<b>Named Entity Recognition (NER)</b> is also called Entity identification where each word is identified in predefined categories like Organization, Place, Person, etc.

## 1. RegEx/Rule-based matching with spaCy(open-source software library for advanced natural language processing)


### Comparison
Compared to using regular expressions on raw text, spaCy’s rule-based matcher engines and components not only let you find the words and phrases you’re looking for — they also give you access to the tokens within the document and their relationships.

This means you can easily access and analyze the surrounding tokens, merge spans into single tokens or add entries to the named entities in doc.ents.

<b>Text:</b> The original word text.<br>
<b>Lemma:</b> The base form of the word.<br>
<b>POS:</b> The simple UPOS part-of-speech tag.<br>
<b>Tag:</b> The detailed part-of-speech tag.<br>
<b>Dep:</b> Syntactic dependency, i.e. the relation between tokens.<br>
<b>Shape:</b> The word shape — capitalization, punctuation, digits.<br>
<b>is alpha:</b> Is the token an alpha character?<br>
<b>is stop:</b> Is the token part of a stop list, i.e. the most common words of the language?<br>

In [ ]:
# Example

import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span, DocBin

TEXTS = [
        'How to preorder the iPhone X', 'iPhone X is coming', 'Should I pay $1,000 for the iPhone X?',
        'The iPhone 8 reviews are here', "iPhone 11 vs iPhone 8: What's the difference?",
        'I need a new phone! Any tips?'
        ]

nlp = spacy.blank("en")
matcher = Matcher(nlp.vocab)

# Two tokens whose lowercase forms match "iphone" and "x"
pattern1 = [{"LOWER": "iphone"}, {"LOWER": "x"}]

# Token whose lowercase form matches "iphone" and a digit
pattern2 = [{"LOWER": "iphone"}, {"IS_DIGIT": True}]

# Add patterns to the matcher and create docs with matched entities
matcher.add("GADGET", [pattern1, pattern2])
docs = []
for doc in nlp.pipe(TEXTS):
    matches = matcher(doc)
    spans = [Span(doc, start, end, label=match_id) for match_id, start, end in matches]
    print(spans)
    doc.ents = spans
    docs.append(doc)

doc_bin = DocBin(docs=docs)
doc_bin.to_disk("./train.spacy")

[iPhone X]
[iPhone X]
[iPhone X]
[iPhone 8]
[iPhone 11, iPhone 8]
[]


In [ ]:
# Example

import pandas as pd
import spacy
from spacy.matcher import Matcher

train_df = pd.read_csv('PII Train Data.csv')
subset_pn = train_df[train_df['Labels'] == 'Phone_number']
print (subset_pn['PII'])

def extract_phone_numbers(text):
    nlp = spacy.load("en_core_web_sm")
    matcher = Matcher(nlp.vocab)

    # Define the phone number pattern
    # 812-942-7968x371
    # 450-241-8512x9924
    # 463.307.7107


    patterns = [[
        {"SHAPE": "ddd"},
        {"ORTH": "-"},
        {"SHAPE": "ddd"},
        {"ORTH": {"IN": ["-", "."]}},
        {"SHAPE": "ddddxdddd"},
    ],
    [
        {"SHAPE": "ddd"},
        {"ORTH": "-"},
        {"SHAPE": "ddd"},
        {"ORTH": {"IN": ["-", "."]}},
        {"SHAPE": "ddddxddd"},
    ]]


# # pattern = [{"ORTH": "("}, {"SHAPE": "ddd"}, {"ORTH": ")"}, {"SHAPE": "ddd"},
# #            {"ORTH": "-", "OP": "?"}, {"SHAPE": "ddd"}]
# pattern = [{"SHAPE": "ddd"}, {"ORTH": "-"}, {"SHAPE": "ddd"},
#            {"ORTH": "-"}, {"SHAPE": "ddddxddd"}]

    # pattern = [
    #     {"SHAPE": "ddd"},
    #     {"ORTH": "-"},
    #     {"SHAPE": "ddd"},
    #     {"ORTH": "-"},
    #     {"SHAPE": "dddd"},
    #     {"ORTH": {"IN": ["x", "X"]}, "OP": "?"},
    #     {"SHAPE": "dddd"},
    # ]

    # Add the pattern to the matcher
    for pattern in patterns:
      matcher.add("PHONE_NUMBER", [pattern])

    # Process the text
    doc = nlp(text)
    print([t.text for t in doc])

    # Find matches in the text
    matches = matcher(doc)

    # Extract phone numbers from the matches
    phone_numbers = []
    for match_id, start, end in matches:
        phone_number = doc[start:end].text
        phone_numbers.append(phone_number)

    return phone_numbers

for index,row in subset_pn.iterrows():
  numbers = extract_phone_numbers(row['Text'])
  print(numbers)

## 2. In-built NER pipelines : NLTK vs spaCy vs Flair vs Transformers(Hugging face)




NLTK - NLTK does NER in two steps. The first step is POS (parts-of-speech) tagging or grammatical tagging, which is followed by chunking to extract the named entities.<br>

spaCy - Bloom embedding and residual CNNs(encode), and a novel transition-based approach to named entity parsing with MLP for inference. BLOOM uses a Transformer architecture composed of an input embeddings layer, 70 Transformer blocks, and an output language-modeling layer <br>
Updated :

Flair - Word embeddings + Bi-directional LSTMs + CRF. Solves word disambiguation to some extent(https://medium.com/analytics-vidhya/practical-approach-of-state-of-the-art-flair-in-named-entity-recognition-46a837e25e6b)<br>
Conditional Random Fields or CRFs are a type of probabilistic graph model that take neighboring sample context into account for tasks like classification. Prediction is modeled as a graphical model, which implements dependencies between the predictions. Graph choice depends on the application, for example linear chain CRFs are popular in natural language processing, whereas in image-based tasks, the graph would connect to neighboring locations in an image to enforce that they have similar predictions.

Transformers -

Dataset - CoNLL - F1 score<br>
NLTK - 0.89<br>
spaCy - 0.87 <br>
Flair - 0.93<br>
BERT - 0.92 <br>
https://paperswithcode.com/sota/named-entity-recognition-ner-on-conll-2003<br>

In general, BERT is probably better for tasks where meaning plays an important role. FLAIR is probably just as good on tasks related to syntax and morphology. Also, the typical advantage of character-level models is their better robustness towards noise (cf. case study in machine translation). There is not much direct comparison, however, there are many indirect clues that suggest so.

FLAIR is evaluated on named entity recognition, chunking, and part-of-speech tagging. All these three tasks rely heavily on syntax. FLAIR reports the F-1 score of 93.09 on the CoNLL-2003 Named Entity Recognition dataset, the same as BERT reports the F1-score of 92.8. (Note, however, that there are BERT-like models that are much better than the original BERT, such as RoBERTa or ALBERT.)

The semantic abilities of pre-trained representations are evaluated in the GLUE and SuperGLUE benchmarks, where BERT is used as a baseline and FLAIR is not in the leaderboards at all. This suggests it would work well. Also, very recent attempts to do character-level pre-trained representations (cf. Charformer, CANINE) struggle to reach good semantic performance, which also suggests that FLAIR as a character-level representation might have problems with semantic-oriented tasks.



## 3. Train your own NER model on top of the spaCy NER model
*Similarly in Flair

1. https://stackoverflow.com/questions/70391922/how-to-i-update-my-trained-space-ner-model-with-new-training-dataset
2. https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718

## 4. BERT-based NER

1. https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270
2. https://towardsdatascience.com/named-entity-recognition-with-bert-in-pytorch-a454405e0b6a
3. https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb
4. https://huggingface.co/dslim/bert-base-NER#:~:text=bert%2Dbase%2DNER%20is%20a,)%20and%20Miscellaneous%20(MISC). However I am not sure if this model can be fine-tuned.

## Resources
1. https://conradobio.medium.com/rule-bases-matching-with-spacy-74469befa163
2. NER Model Evaluation - https://towardsdatascience.com/a-pathbreaking-evaluation-technique-for-named-entity-recognition-ner-93da4406930c
3. NLTK vs spaCy vs Flair - https://medium.com/quantrium-tech/top-3-packages-for-named-entity-recognition-e9e14f6f0a2a
4. Entity disambiguation - https://towardsdatascience.com/improving-named-entity-disambiguation-using-entity-relatedness-within-wikipedia-92f400ee5994
5. NER internal architecture - https://medium.com/@sapphireduffy/is-flair-a-suitable-alternative-to-spacy-6f55192bfb01
6. https://medium.com/thecyphy/training-custom-ner-model-using-flair-df1f9ea9c762

# In-Depth

### Tokenization
Tokenization is used in natural language processing to split paragraphs and sentences into smaller units that can be more easily assigned meaning.

1. Word tokenization : It takes natural breaks, like pauses in speech or spaces in text, and splits the data into its respective words using delimiters (characters like ‘,’ or ‘;’ or ‘“,”’). While this is the simplest way to separate speech or text into its parts, it does come with some drawbacks. It’s difficult for word tokenization to separate unknown words or Out Of Vocabulary (OOV) words. This is often solved by replacing unknown words with a simple token that communicates that a word is unknown. This is a rough solution, especially since 5 ‘unknown’ word tokens could be 5 completely different unknown words or could all be the exact same word.

2. Character tokenization : Character tokenization doesn’t have the same vocabulary issues as word tokenization as the size of the ‘vocabulary’ is only as many characters as the language needs. For English, for example, a character tokenization vocabulary would have about 26 characters. While character tokenization solves OOV issues, it isn‘t without its own complications. By breaking even simple sentences into characters instead of words, the length of the output is increased dramatically. With word tokenization, our previous example “what restaurants are nearby” is broken down into four tokens. By contrast, character tokenization breaks this down into 24 tokens, a 6X increase in tokens to work with. Character tokenization also adds an additional step of understanding the relationship between the characters and the meaning of the words.

3. Sub-word tokenization : if the NLP model was using sub word tokenization, it would be able to separate the word into an ‘unknown’ token and an ‘ing’ token. From there it can make valuable inferences about how the word functions in the sentence. But what information can a machine gather from a single suffix? The common ‘ing’ suffix, for example, functions in a few easily defined ways. It can form a verb into a noun, like the verb ‘build’ turned into the noun ‘building’. It can also form a verb into its present participle, like the verb ‘run’ becoming ‘running.’ If an NLP model is given this information about the ‘ing’ suffix, it can make several valuable inferences about any word that uses the sub word ‘ing.’  If ‘ing’ is being used in a word, it knows that it is either functioning as a verb turned into a noun, or as a present verb. This dramatically narrows down how the unknown word, ‘machinating,’ may be used in a sentence.

### Word embeddings/vectors

As we learned already, we create vectors from tokens. These vectors are numerical representations of the meaning of a token and how it relates to other tokens in our vocabulary.
However, the model must first learn these representations. Initially, the vectors are randomly initialized, but through training, they are adapted to incorporate some meaning.
During training, these numerical representations, similar to weights in a neural network, are adjusted by the model to more accurately represent the meaning of a token and its relationship to other tokens in our vocabulary.

For example, The vectors for the words “King” and “Man” may be similar, as might the vectors for “Queen” and “Woman.” These vectors also have certain properties that can be useful for training language models.

For example, you can subtract the vector for “Man” from the vector for “King” and add the vector for “Woman” to get the vector for “Queen.” These properties allow the model to understand the different meanings of words.

For example, a word vector for the word “queen” might have high values for dimensions representing “femininity” and “royalty” and low values for dimensions representing “masculinity.”


### From Text to Vectors

Creating a high-quality corpus and training a tokenizer can be time-consuming and may not always produce the best results.

One alternative is to use pre-trained tokenizers, which have already been trained on a large dataset and can understand the meanings of words and sentences without additional training.


#### How Bert Tokenizer works, and why BERT over Word2vec, Glove, ELMo?

1. We use the pre-trained bert-base-uncased tokenizer to tokenize a sample sentence. Then, we will use the convert_tokens_to_ids function of the tokenizer to convert the tokens to their corresponding numerical values.
2. The encode function is similar to convert_tokens_to_ids, but it also includes special tokens such as <BOS>(beginning of the sequence) and <EOS>(end of the sequence). These special tokens help the model understand where a sequence starts and ends.
3. The encode function translates the token with id 101 to the special [CLS] token, which represents the beginning of the sequence <BOS> token. The token with id 102 is the end of the sequence (<EOS>) token.
5. These tokens are mapped to dense vectors in the embedding space, which encode the meanings of the words and their relationships to other words.
6. You can use the embeddings.word_embeddings method to return its word vector
7. The returned word vector has a size of 768 dimensions, the same as the BERT model.

https://www.quora.com/What-are-the-main-differences-between-the-word-embeddings-of-ELMo-BERT-Word2vec-and-GloVe


### Model

To give these tokens meaning, a deep learning model, often a transformer model, is trained on these vectors. This allows the model to understand the meaning of words and how they relate to each other.

The goal of this process is to enable NLP models to understand the meaning and semantics of different words and their context within a sentence or text.

#### Transformers --> BERT

1. The BERTBase model uses 12 layers of transformers block with a hidden size of 768 and number of self-attention heads as 12 and has around 110M trainable parameters.
2. Unlike RNNs where the decoder hidden state has access only to the last hidden state, here it has access to all the inputs' encoder representation. On top of that, it also ensure how much attention to give to each.
q,k,v - What we want to do is take query q and find the most similar key k, by doing a dot product for q and k. The closest query-key product will have the highest value, followed by a softmax that will drive the q.k with smaller values close to 0 and q.k with larger values towards 1. This softmax distribution is multiplied with v. The value vectors multiplied with ~1 will get more attention while the ones ~0 will get less. The sizes of these q, k and v vectors are referred to as “hidden size” by various implementations.
3. The three kinds of Attention possible in a model:
Encoder-Decoder Attention: Attention between the input sequence and the output sequence.
Self attention in the input sequence: Attends to all the words in the input sequence.
Self attention in the output sequence: One thing we should be wary of here is that the scope of self attention is limited to the words that occur before a given word. This prevents any information leaks during the training of the model. This is done by masking the words that occur after it for each step. So for step 1, only the first word of the output sequence is NOT masked, for step 2, the first two words are NOT masked and so on.
4. Positional information
5. BERT’s key technical innovation is applying the bidirectional training of Transformer, a popular attention model, to language modelling.
6. A language model which is bidirectionally trained can have a deeper sense of language context and flow than single-direction language models.
7. Since BERT’s goal is to generate a language model, only the encoder mechanism is necessary.
8. BERT uses two training strategies: MLM + NSP
9. To finetune BERT : In Named Entity Recognition (NER), the software receives a text sequence and is required to mark the various types of entities (Person, Organization, Date, etc) that appear in the text. Using BERT, a NER model can be trained by feeding the output vector of each token into a classification layer that predicts the NER label.